In [10]:
import numpy as np
from utils import *
import pandas as pd
from sklearn.metrics import confusion_matrix

data = pd.read_table("data/dog_crowd.txt", header=None)
truth = pd.read_table("data/dog_truth.txt", header=None)


In [11]:
truth = truth.drop_duplicates()

In [12]:
n, m, k = np.max(np.array(data), axis=0)
print(n, m, k)


807 109 4


In [13]:
labels = transform_data(data)


In [14]:
labels

array([[ 3,  2, -1, ..., -1, -1, -1],
       [ 2, -1, -1, ..., -1, -1, -1],
       [ 3,  2,  1, ..., -1, -1,  1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int64)

In [15]:
groups = np.array([i % 3 for i in range(m)])


In [16]:
C_estimated = get_confusion_matrix(k, labels, groups=groups, sym=False)


In [17]:
i = 50
truth_new = truth[truth[0].isin(data[data[1] == i+1][0])]
pred_new = data[data[1] == i+1][data[data[1] == i+1][0].isin(truth_new[0])]


In [18]:
C = confusion_matrix(pred_new[2], truth_new[1])


In [19]:
np.round(C / np.sum(C, axis=0)[np.newaxis, :], 5)


array([[0.71429, 0.3    , 0.     , 0.     ],
       [0.28571, 0.7    , 0.     , 0.     ],
       [0.     , 0.     , 0.61111, 0.07143],
       [0.     , 0.     , 0.38889, 0.92857]])

In [20]:
np.round(C_estimated[i], 5)


array([[8.8915e-01, 1.6008e-01, 0.0000e+00, 1.5360e-02],
       [1.0804e-01, 8.3902e-01, 1.4710e-02, 0.0000e+00],
       [2.8100e-03, 9.1000e-04, 7.9719e-01, 1.7400e-02],
       [0.0000e+00, 0.0000e+00, 1.8809e-01, 9.6724e-01]])

In [56]:
def get_true_confusion_matrix(data, truth, normalize=True):
    truth = truth.drop_duplicates()
    n, m, k = np.max(np.array(data), axis=0)
    C = np.zeros((m, k, k))
    for i in range(m):
        truth_new = truth[truth[0].isin(data[data[1] == i+1][0])]
        pred_new = data[data[1] == i +
                        1][data[data[1] == i+1][0].isin(truth_new[0])]
        Ci = confusion_matrix(pred_new[2], truth_new[1], labels=[
                              i+1 for i in range(k)])
        if normalize:
            Ci = Ci.astype(float)
            colsums = np.sum(Ci, axis=0)
            Ci[:, colsums != 0] /= colsums[np.newaxis, colsums != 0]
        C[i, :, :] = Ci
    return C


In [59]:
def confusion_matrix_loss(C_estimated,C_true):
    return np.mean((C_estimated-C_true)**2)


In [43]:
truth_new[1].drop_duplicates().sort_values()

80     1
580    2
270    3
350    4
Name: 1, dtype: int64

In [57]:
C_true = get_true_confusion_matrix(data, truth)


In [58]:
C_true

array([[[0.85294118, 0.16666667, 0.        , 0.07142857],
        [0.14705882, 0.83333333, 0.        , 0.        ],
        [0.        , 0.        , 0.71153846, 0.14285714],
        [0.        , 0.        , 0.28846154, 0.78571429]],

       [[0.35      , 0.14285714, 0.        , 0.        ],
        [0.6       , 0.82142857, 0.        , 0.        ],
        [0.05      , 0.03571429, 0.47058824, 0.24137931],
        [0.        , 0.        , 0.52941176, 0.75862069]],

       [[0.37313433, 0.23684211, 0.        , 0.        ],
        [0.58208955, 0.75      , 0.01123596, 0.01020408],
        [0.01492537, 0.01315789, 0.88764045, 0.60204082],
        [0.02985075, 0.        , 0.1011236 , 0.3877551 ]],

       ...,

       [[1.        , 0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        ],
        [0.        , 0.        , 0.5       , 0.5       ],
        [0.        , 0.        , 0.5       , 0.5       ]],

       [[1.        , 0.        , 0.        , 0.    

In [65]:
confusion_matrix_loss(get_confusion_matrix(
    k, labels, groups=groups, sym=True), C_true)


0.03289894038021084